In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# LOADING THE DATASETS

In [1]:
train_img=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [1]:
train_img.tail(10)

In [1]:
Y=train_img['label']

In [1]:
X=train_img.drop(labels='label',axis=1)
X

In [1]:
Y

In [1]:
g = sns.countplot(Y)

Y.value_counts()

# CHECKING FOR NULL VALUES

In [1]:
X.isnull().any().describe()

In [1]:
test.isnull().any().describe()

So there are no null values.


#     # NORMALISING THE PIXEL VALUES

In [1]:
X = X/ 255.0
test= test / 255.0

In [1]:
X

# Train and test images (28px x 28px) has been stock into pandas.Dataframe as 1D vectors of 784 values. We reshape all data to 28x28x1 3D matrices.

# Keras requires an extra dimension in the end which correspond to channels. MNIST images are gray scaled so it use only one channel. For RGB images, there is 3 channels, we would have reshaped 784px vectors to 28x28x3 3D matrices.

# If one component of shape is the special value -1, the size of that dimension is computed so that the total size remains constant. In particular, a shape of [-1] flattens into 1-D. At most one component of shape can be -1.

In [1]:
X = X.values.reshape(-1,28,28,1)
test= test.values.reshape(-1,28,28,1)

In [1]:
X

In [1]:
Y

# SPLITTING THE DATASET

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [1]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.1,random_state=42)

In [1]:
x_train.shape

In [1]:
x_test.shape

In [1]:
from tensorflow.keras import datasets, layers, models

In [1]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization

# CNN MODEL

In [1]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [1]:
model.summary()

In [1]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=25)

In [1]:
test_loss, test_acc = model.evaluate(x_test, y_test)

# TEST ACCURACY

In [1]:
print(test_acc)

In [1]:
print(test_loss)

In [1]:
test

In [1]:
results=model.predict(test)

In [1]:
results.shape

In [1]:
results

In [1]:
classes= model.predict_classes(test)
classes

In [1]:
results = pd.Series(classes,name="Label")

# SUBMISSION

In [1]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission

In [1]:
submission.to_csv("submit5.csv",index=False)